<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/assignment7_wordembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
import os 
os.environ['OMP_NUM_THREADS'] = '64'

In [ ]:
import sys
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install --upgrade gensim -qq
    !pip install -U spacy -qq


    basepath = '/content/drive/MyDrive/NLP/assignment7'
    sys.path.append('/content/drive/MyDrive/NLP/custom-functions')

Mounted at /content/drive


In [ ]:
import random
import spacy
import textwrap
import joblib
from pathlib import Path
import numpy as np
import gensim
import gensim.downloader as loader
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import itertools

In [ ]:
import os
os.cpu_count()

2

In [ ]:
print(f'spacy-version: {spacy.__version__}  || gensim-version{ gensim.__version__}')

spacy-version: 3.5.2  || gensim-version4.3.1


In [ ]:
#!python3 --version

In [ ]:
if gensim.models.doc2vec.FAST_VERSION > -1:
    print("Fast gensim version is being used.")
else:
    print("Slow gensim version is being used.")

Fast gensim version is being used.


In [ ]:
folder = Path(basepath)/'WordEmbeddings'

In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.9/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/NLP/custom-functions']

In [ ]:
stackExchange_cleaned_text_file = folder / 'df_raw_small_hw.joblib'    #Folder path was set for accessing the file

In [ ]:
#Load the file 
stackExchange_cleaned_text = joblib.load(stackExchange_cleaned_text_file)

In [ ]:
type(stackExchange_cleaned_text)

pandas.core.frame.DataFrame

In [ ]:
stackExchange_cleaned_text.shape

(438813, 4)

In [ ]:
stackExchange_cleaned_text.columns

Index(['Unnamed: 0', 'Id', 'Title', 'Body'], dtype='object')

In [ ]:
stackExchange_cleaned_text = stackExchange_cleaned_text.drop(columns=['Unnamed: 0'])

In [ ]:
stackExchange_cleaned_text.head(5)

,Id,Title,Body
3595496,7254900,Identify un-activated servers,<p>I've been on a bit of a roll this week and ...
1929804,4905251,How to abort the loading of an external HTML r...,"<p>I'm writing a JavaScript widget, which is i..."
2513704,6173108,Xcode: how do you make it to where clicking on...,<p>I know how to make a button in Xcode with I...
3794169,7453573,Common WQL Monitoring Queries,<p>What WQL queries would you use for monitori...
4341677,8001081,Loading Native Managed and C++ DLL within IIS ...,<p>I have WCF service developed in C# for with...


In [ ]:
stackExchange_sentences = [text.split() for text in stackExchange_cleaned_text['Body']]
#len(stackExchange_cleaned_text)
#print(stackExchange_cleaned_text)
#stackExchange_preprocess_sent = []
#for text in stackExchange_cleaned_text['Body']:
  #sentence = preprocess(text)
  #stackExchange_preprocess_sent.append(sentence)

In [ ]:
type(stackExchange_sentences)

list

In [ ]:
len(stackExchange_sentences)

438813

In [ ]:
print(stackExchange_sentences[0])

["<p>I've", 'been', 'on', 'a', 'bit', 'of', 'a', 'roll', 'this', 'week', 'and', 'have', 'deployed', 'around', '30', 'fresh', 'installs', 'of', 'Windows', 'Server', '2008', 'R2', 'using', 'our', 'Datacentre', 'license,', 'inside', 'a', 'VMWare', 'environment.</p>', '<p>However,', 'what', "I've", 'gone', 'and', 'forgotten', 'as', 'part', 'of', 'the', 'setup', 'routine', 'is', 'to', 'activate', 'the', 'servers.</p>', '<p>The', 'last', 'thing', 'I', 'want', 'is', 'next', 'month', 'to', 'find', 'out', 'that', 'our', 'clients', "can't", 'log', 'into', 'our', 'terminal', 'servers', 'because', "I've", 'forgotten', 'to', 'activate', 'them,', 'or', 'for', 'them', 'to', 'see', '"You', 'have', 'been', 'a', 'victim', 'of', 'software', 'piracy"', 'pop', 'up', 'on', 'their', 'desktop.</p>', '<p>Is', 'there', 'any', 'script/dirty', 'hacks', 'that', 'I', 'can', 'use', 'to', 'quickly', 'scan', 'the', 'network', 'for', 'un-activated', 'machines?', "I'm", 'guessing', 'it', 'will', 'probably', 'be', 'a', '

In [ ]:
model_stackExchange_CBOW = Word2Vec(sentences=stackExchange_sentences,epochs=10, vector_size=300, window=10, min_count=5, workers = 32, sg=0)
#model_stackExchange_CBOW.build_vocab(sentences=stackExchange_sentences)

#model_stackExchange_CBOW.train(sentences=stackExchange_sentences, total_examples=len(stackExchange_sentences), epochs=10)


In [ ]:
type(model_stackExchange_CBOW)

gensim.models.word2vec.Word2Vec

In [ ]:
model_stackExchange_CBOW.vector_size

150

In [ ]:
model_stackExchange_CBOW.wv.vectors.shape  #There are 356393 unique words in the corpus and each word is represented by 150 dimensional vector

(356393, 150)

In [ ]:
model_stackExchange_CBOW.wv.save(str(folder/'model_stackExchange_CBOW.bin')) #Saved the file to google drive folder

In [ ]:
model_stackExchange_CBOW_vectors = KeyedVectors.load(str(folder/'model_stackExchange_CBOW.bin'))

In [ ]:
try:
  model_stackExchange_CBOW_vectors['science']
except:
  print(f'word science is not in vocab of model')

In [ ]:
model_stackExchange_CBOW_vectors['science']

array([ 0.40962756,  2.6622126 ,  0.20537792,  0.87750244,  1.4648224 ,
        0.14702705, -0.8002696 ,  1.8798313 , -1.2703792 ,  1.6898727 ,
       -0.60029346,  2.605713  ,  0.04700831, -1.7430087 ,  1.0076667 ,
        0.14075007,  0.02483106,  0.63657176,  1.6531366 ,  1.4334024 ,
        0.55768216, -0.12525578,  1.4372687 ,  0.31104997, -1.0186431 ,
        0.71017826, -1.7429302 , -1.2922642 , -1.5820191 ,  1.5602354 ,
       -0.7226889 , -2.2022302 , -2.8540323 , -1.4683979 ,  2.256781  ,
       -2.7449193 ,  1.0760417 ,  3.564089  ,  1.8089902 ,  1.2642685 ,
        1.1811726 ,  1.030965  ,  1.493262  , -0.76820266,  1.2845331 ,
       -1.2838773 ,  0.28420436, -0.28382382,  2.1914089 , -0.05275181,
        1.9114656 ,  0.6627896 ,  0.8890082 , -1.2365764 , -0.19270048,
       -0.95815593,  0.24627459,  2.1811895 , -0.9778044 ,  1.292354  ,
       -3.0625806 , -0.37509164, -0.98846424, -0.46962464,  1.350784  ,
       -0.53707457,  0.22979483, -0.17672738,  0.5801698 , -0.60

In [ ]:
vocab_stackExchange_cbow=model_stackExchange_CBOW_vectors.index_to_key

In [ ]:
type(vocab_stackExchange_cbow)

list

In [ ]:
vocab_stackExchange_cbow[0:5]

['the', 'to', 'I', 'a', '=']

In [ ]:
model_stackExchange_CBOW_vectors.most_similar('beautiful',topn=10)

[('wonderful', 0.595547080039978),
 ('cool', 0.5950064659118652),
 ('nice', 0.5883001685142517),
 ('famous', 0.5629765391349792),
 ('slick', 0.5291317701339722),
 ('nice,', 0.512283205986023),
 ('jargon', 0.5087358951568604),
 ('fun', 0.5054478645324707),
 ('loved', 0.49093788862228394),
 ('beautiful,', 0.4891890585422516)]

In [ ]:
print(f"similarity between airline and car is : {model_stackExchange_CBOW_vectors.similarity('slick','fun')}")

similarity between airline and car is : 0.3545458912849426


In [ ]:
for i in  random.choices(range(1,3000),k=20):
    print(vocab_stackExchange_cbow[i])

c
below.</p>
question.</p>
&lt;/ul&gt;
e.printStackTrace();
situation
keep
best
==
readonly
simple
pixels
works.
slide
drive
receives
TABLE
longer
operation
<pre><code>for


In [ ]:
model_stackExchange_CBOW_vectors.doesnt_match(['printStackTrace','woman','readonly'])

'woman'